# Extracting Knowledge Graphs with LLMs

You can use LLMs to extract graph data in order to create a Knowledge Graph for the system to request. However, if you want to implement Knowledge Graph RAG for business operations, we advise you to create the graph yourself instead of relying on LLMs, they can introduce many noise and it is always better to understand the data you want to query.

In [3]:
import dspy
from hybridagi.readers import TextReader
from hybridagi.core.pipeline import Pipeline
from hybridagi.embeddings import SentenceTransformerEmbeddings
from hybridagi.memory.integration.falkordb import FalkorDBFactMemory
from hybridagi.modules.embedders import FactEmbedder
from hybridagi.modules.extractors import LLMFactExtractor
from hybridagi.modules.deduplicators import EntityDeduplicator
from hybridagi.modules.splitters import DocumentSentenceSplitter

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n", "\n---", "\n\nContext:"])
dspy.configure(lm=lm)

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384,
)

reader = TextReader()

input_docs = reader("data/SynaLinks_presentation.md")

pipeline = Pipeline()

pipeline.add("chunk_docs", DocumentSentenceSplitter(
    method = "word",
    chunk_size = 100,
    chunk_overlap = 0,
    separator = " ",
))
pipeline.add("extract_facts", LLMFactExtractor())
pipeline.add("deduplicate_entities", EntityDeduplicator(method="exact"))
pipeline.add("embed_facts", FactEmbedder(embeddings=embeddings))

output_facts = pipeline(input_docs)

fact_memory = FalkorDBFactMemory(
    index_name="synalinks_graph", 
    graph_index="synalinks_graph",
    embeddings=embeddings,
    wipe_on_start=True,
)

fact_memory.update(output_facts)


100%|██████████| 22/22 [00:00<00:00, 95.82it/s]
